<a href="https://colab.research.google.com/github/misbakhanum/SCT_ML_4/blob/main/SCT_T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving archive (1).zip to archive (1).zip


In [2]:
import zipfile

# Unzip the uploaded file
file_name = '/content/archive (1).zip'  # Replace with your actual uploaded file name
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall('/content/hands_folder')

In [8]:
import os

# Check the contents of the hands_folder directory
for root, dirs, files in os.walk('/content/hands_folder'):
    level = root.replace('/content/hands_folder', '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f'{indent}{os.path.basename(root)}/')
    for f in files:
        print(f'{indent}    {f}')


Streaming output truncated to the last 5000 lines.
                960.jpg
                1128.jpg
                911.jpg
                1188.jpg
                1171.jpg
                980.jpg
                1038.jpg
                1015.jpg
                959.jpg
                1030.jpg
                997.jpg
                919.jpg
                1044.jpg
                1160.jpg
                1116.jpg
                990.jpg
                1064.jpg
                1080.jpg
                1172.jpg
                972.jpg
                962.jpg
                1043.jpg
                987.jpg
                1193.jpg
                1157.jpg
                1190.jpg
                1110.jpg
                1065.jpg
                1155.jpg
                929.jpg
                998.jpg
                996.jpg
                1181.jpg
                1122.jpg
                1056.jpg
                917.jpg
                908.jpg
                930.jpg
               

In [17]:
# Function to load images recursively from folder (filter only '.jpg' files)
def load_images_from_folder(folder, label):
    images = []
    labels = []
    valid_extensions = ('.jpg', '.jpeg', '.png')  # Add more extensions if needed
    for root, dirs, files in os.walk(folder):  # Recursively search all subdirectories
        for filename in files:
            if filename.lower().endswith(valid_extensions):  # Check for valid image extensions
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
                if img is not None:  # Check if the image is loaded successfully
                    img = cv2.resize(img, (64, 64))  # Resize to 64x64 pixels
                    images.append(img.flatten())  # Flatten the image into a 1D array
                    labels.append(label)
                else:
                    print(f"Failed to load image: {img_path}")
            else:
                print(f"Skipped non-image file: {filename}")
    return images, labels

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Function to load images recursively from folder (filter only '.jpg' and other image files)
def load_images_from_folder(folder):
    images = []
    labels = []
    valid_extensions = ('.jpg', '.jpeg', '.png')  # Add more extensions if needed

    # Iterate through the folder structure
    for label, subfolder in enumerate(os.listdir(folder)):
        subfolder_path = os.path.join(folder, subfolder)  # Get path to the subfolder
        if os.path.isdir(subfolder_path):  # Check if it's a directory
            for root, dirs, files in os.walk(subfolder_path):
                for filename in files:
                    if filename.lower().endswith(valid_extensions):  # Check for valid image extensions
                        img_path = os.path.join(root, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
                        if img is not None:  # Check if the image is loaded successfully
                            img = cv2.resize(img, (64, 64))  # Resize to 64x64 pixels
                            images.append(img.flatten())  # Flatten the image into a 1D array
                            labels.append(label)
                        else:
                            print(f"Failed to load image: {img_path}")
                    else:
                        print(f"Skipped non-image file: {filename}")

    return np.array(images), np.array(labels)

# Load images from the specified folder
folder_path = '/content/hands_folder'  # Replace with your folder path
X, y = load_images_from_folder(folder_path)

# Normalize the images
X = X.astype('float32') / 255.0  # Normalize pixel values to range [0, 1]
num_classes = len(np.unique(y))  # Number of classes

# One-hot encode the labels
y = to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data for the CNN model (samples, height, width, channels)
X_train = X_train.reshape(-1, 64, 64, 1)
X_test = X_test.reshape(-1, 64, 64, 1)

# Model Creation
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Number of classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Training
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

# Plot Training History
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save the model
model.save('hand_gesture_model.h5')

# Function to make predictions
def predict_gesture(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    img = img.flatten() / 255.0  # Normalize
    img = img.reshape(1, 64, 64, 1)  # Reshape for the model
    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    class_label = class_index  # Adjust as per your class labels
    return class_label

# Test the prediction function
test_image_path = 'test_image.jpg'  # Replace with your test image path
predicted_gesture = predict_gesture(test_image_path)
print(f'The predicted gesture is: {predicted_gesture}')


Epoch 1/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.7688 - loss: 0.5012